**Устанавливаем все необходимые пакеты Python**

In [1]:
#Для расчета трассы полета
!pip install pyorbital
#Для получения TLE файла из базы
!pip install spacetrack

#Для работы с шейп-файлами
!pip install pyshp
!pip install shapely
!pip install shapefile

#Для работы с геоданными
!pip install geopandas
!pip install contextily
!pip install fiona

#Для создания интерактивной карты
!pip install folium

ERROR: Could not find a version that satisfies the requirement shapefile (from versions: none)
ERROR: No matching distribution found for shapefile


Импортируем библиотеки

In [2]:
#Библиотеки для построения трассировки КА
from datetime import datetime, date, timedelta
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
from pyorbital.orbital import Orbital

#Библиотеки для работы с шейпфайлами (shapefiles)
import shapefile
import fiona
from shapely.geometry import Point, Polygon, mapping

#Библиотеки для математических вычислений и моделирования
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import contextily as ctx
import pandas as pd
import csv
import math

#библиотека для всплывающего окна(ЮзерФрендли)
import tkinter as tk
from tkinter import filedialog, messagebox
import os

#Библиотека folium для создания интерактивной карты
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
print('All installed and imported!')

All installed and imported!


функция по созданию диалогового окна для взаимодействия с пользователем

In [3]:
def create_input_dialog():
    """Создает и возвращает диалоговое окно для ввода параметров"""
    root = tk.Tk()
    root.title("Ввод параметров")
    root.resizable(False, False)
    
    # Словарь для хранения виджетов и значений
    entries = {}
    values = {
        'papka': tk.StringVar(value=os.getcwd()),
        'USERNAME': tk.StringVar(),
        'PASSWORD': tk.StringVar(),
        'nomer': tk.StringVar(),
        'interval': tk.StringVar(value="30"),
        'vitkov': tk.StringVar(value="24"),
        'SK': tk.StringVar(value="3043"),
        'year': tk.StringVar(value="2025"),
        'mounth': tk.StringVar(value="1"),
        'day': tk.StringVar(value="1"),
        'max_ugol_obz': tk.StringVar(value="90"),
        'min_ugol_obz': tk.StringVar(value="-90"),
        'max_ugol_viz': tk.StringVar(value="50"),
        'min_ugol_viz': tk.StringVar(value="20")
    }
    
    # Функция для выбора папки
    def browse_folder():
        folder = filedialog.askdirectory()
        if folder:
            values['papka'].set(folder)
    
    # Функция проверки и сохранения данных
    def submit():
        errors = []
        # Проверка числовых полей
        numeric_fields = [
            'nomer', 'interval', 'vitkov', 'SK',
            'year', 'mounth', 'day',
            'max_ugol_obz', 'min_ugol_obz',
            'max_ugol_viz', 'min_ugol_viz'
        ]
        
        for field in numeric_fields:
            try:
                int(values[field].get())
            except ValueError:
                errors.append(f"Некорректное значение в поле '{field_labels[field]}'")
        
        if not values['papka'].get():
            errors.append("Путь к папке не указан")
        
        if errors:
            messagebox.showerror("Ошибки ввода", "\n".join(errors))
            return
        
        root.destroy()
    
    # Поля и их подписи
    field_labels = {
        'papka': "Рабочая папка:",
        'USERNAME': "Имя пользователя spacetrack.org:",
        'PASSWORD': "Пароль spacetrack.org:",
        'nomer': "NORAD ID КА:",
        'interval': "Временной интервал (секунды):",
        'vitkov': "Количество частей трассы:",
        'SK': "Система координат(метрическая):",
        'year': "Год:",
        'mounth': "Месяц:",
        'day': "День:",
        'max_ugol_obz': "Макс. широта наблюдения:",
        'min_ugol_obz': "Мин. широта наблюдения:",
        'max_ugol_viz': "Макс. угол визирования:",
        'min_ugol_viz': "Мин. угол визирования:"
    }
    
    # Создание элементов формы
    for i, field in enumerate(field_labels):
        # Подпись
        lbl = tk.Label(root, text=field_labels[field], anchor='e')
        lbl.grid(row=i, column=0, padx=5, pady=5, sticky='e')
        
        # Поле ввода
        if field == 'PASSWORD':
            entry = tk.Entry(root, textvariable=values[field], show="*")
        else:
            entry = tk.Entry(root, textvariable=values[field])
        
        entry.grid(row=i, column=1, padx=5, pady=5, sticky='we')
        entries[field] = entry
        
        # Кнопка выбора папки
        if field == 'papka':
            btn_browse = tk.Button(root, text="Обзор", command=browse_folder)
            btn_browse.grid(row=i, column=2, padx=5, pady=5)
    
    # Кнопки управления
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=len(field_labels), column=0, columnspan=3, pady=10)
    
    btn_submit = tk.Button(btn_frame, text="OK", command=submit)
    btn_submit.pack(side=tk.LEFT, padx=10)
    
    btn_cancel = tk.Button(btn_frame, text="Отмена", command=root.destroy)
    btn_cancel.pack(side=tk.LEFT, padx=10)
    
    # Настройка размеров колонок
    root.columnconfigure(0, weight=0)
    root.columnconfigure(1, weight=1)
    root.columnconfigure(2, weight=0)
    
    root.mainloop()
    
    # Возвращаем результаты после закрытия окна
    if all(values[field].get() for field in values):
        return {k: v.get() for k, v in values.items()}
    return None

создание диалогового окна

In [4]:
if __name__ == "__main__":
    # Вызываем диалоговое окно
    input_data = create_input_dialog()
    papka = input_data['papka']
    USERNAME = input_data['USERNAME']
    PASSWORD = input_data['PASSWORD']
    nomer = int(input_data['nomer'])
    interval = int(input_data['interval'])
    vitkov = int(input_data['vitkov'])
    SK = int(input_data['SK'])
    year = int(input_data['year'])
    mounth = int(input_data['mounth'])
    day = int(input_data['day'])
    max_ugol_obz = int(input_data['max_ugol_obz'])
    min_ugol_obz = int(input_data['min_ugol_obz'])
    max_ugol_viz = int(input_data['max_ugol_viz'])
    min_ugol_viz = int(input_data['min_ugol_viz'])
    
    # Вычисляем шаг
    shag = int(24 / vitkov * 3600)

Функция получения начальных условий в формате TLE файла из базы данных spacetrack.org

In [5]:
def get_spacetrack_tle (sat_id, start_date, end_date, username, password, latest=False):
    st = SpaceTrackClient(identity=username, password=password) #идентификация пользователя
    if not latest:
        daterange = op.inclusive_range(start_date, end_date) #промежуток дат, на котором нужны TLE
        data = st.tle(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle', epoch = daterange) #начальные условия на заданном промежутке в прошлом
        print(daterange)
    else:
        data = st.tle_latest(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle') #актуальные начальные условия на сегодняшнюю дату
        print(date.today()) #дата актуального TLE

    if not data:
        return 0, 0

    f = open(papka + '/tle.txt', 'w')
    f.write(data[0:69] + '\n')
    f.write(data[70:139])
    f.close()
    return 'Txt create'

Функция расчёта подспутниковых точек на интервал времени в районе заданного дня

In [6]:
def create_orbital_track_shapefile_for_day (sat_id, track_day, start_time, end_time, step_seconds, output_shapefile, input_txt):
    # Для начала получаем TLE
    # Если запрошенная дата наступит в будущем, то запрашиваем самые последний набор TLE
    f = open(input_txt, 'r')
    tle_1 = f.readline()
    tle_2 = f.readline()

    # Создаём экземляр класса Orbital
    orb = Orbital("N", line1=tle_1, line2=tle_2)

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape = shapefile.Writer(output_shapefile)

    # Добавляем поля - идентификатор, время, широту и долготу
    # N - целочисленный тип, C - строка, F - вещественное число
    # Для времени придётся использовать строку, т.к. нет поддержки формата "дата и время"
    track_shape.field('ID','N', 40)
    track_shape.field('TIME','C', 40)
    track_shape.field('LAT','F', 40)
    track_shape.field('LON','F', 40)

    # Объявляем счётчики, i для идентификаторов, minutes для времени
    i = 0
    start_seconds = int((start_time - datetime(start_time.year, start_time.month, start_time.day, 0, 0, 0)).total_seconds())
    for seconds in range(start_seconds, start_seconds + int((end_time - start_time).total_seconds()), step_seconds):
        utc_hour = seconds // 3600
        utc_minutes = (seconds // 60) - (utc_hour * 60)
        utc_seconds = seconds - (utc_minutes * 60) - (utc_hour * 3600)

        # Сформируем строку для атрибута
        utc_string = str(datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds))
        # И переменную с временем текущего шага в формате datetime
        utc_time = datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds)

        # Считаем положение спутника
        lon, lat, alt = orb.get_lonlatalt(utc_time)

        # Создаём в шейп-файле новый объект
        # Определеяем геометрию
        track_shape.point(lon,lat)
        # и атрибуты
        track_shape.record(i, utc_string, lat, lon)

        # Не забываем про счётчики
        i += 1

    # Вне цикла нам осталось записать созданный шейп-файл на диск.
    # Т.к. мы знаем, что координаты положений ИСЗ были получены в WGS84
    # можно заодно создать файл .prj с нужным описанием

    try:
        prj = open("%s.prj" % output_shapefile.replace('.shp',''), "w")
        # Создаем переменную с описанием EPSG:4326 (WGS84)
        wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]'
        # Записываем её в файл .prj
        prj.write(wgs84_wkt)
        # И закрываем его
        prj.close()
        # Функцией save также сохраняем и сам шейп.
        track_shape.close()
        print('End of code!')
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

Функция расчёта подспутниковых точек на определённых широтах(зоны интересов) для сокращения вычислительной нагрузки

In [7]:
def create_orbital_track_shapefile_north (sat_id, track_day, start_time, end_time, step_seconds, output_shapefile, min_ugol, max_ugol, input_txt):
    # Для начала получаем TLE
    # Если запрошенная дата наступит в будущем, то запрашиваем самые последний набор TLE
    f = open(input_txt, 'r')
    tle_1 = f.readline()
    tle_2 = f.readline()

    # Создаём экземляр класса Orbital
    orb = Orbital("N", line1=tle_1, line2=tle_2)

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape = shapefile.Writer(output_shapefile)

    # Добавляем поля - идентификатор, время, широту и долготу
    # N - целочисленный тип, C - строка, F - вещественное число
    # Для времени придётся использовать строку, т.к. нет поддержки формата "дата и время"
    track_shape.field('ID','N', 40)
    track_shape.field('TIME','C', 40)
    track_shape.field('LAT','F', 40)
    track_shape.field('LON','F', 40)

    # Объявляем счётчики, i для идентификаторов, minutes для времени
    i = 0
    start_seconds = int((start_time - datetime(start_time.year, start_time.month, start_time.day, 0, 0, 0)).total_seconds())
    for seconds in range(start_seconds, start_seconds + int((end_time - start_time).total_seconds()), step_seconds):
        utc_hour = seconds // 3600
        utc_minutes = (seconds // 60) - (utc_hour * 60)
        utc_seconds = seconds - (utc_minutes * 60) - (utc_hour * 3600)

        # Сформируем строку для атрибута
        utc_string = str(datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds))
        # И переменную с временем текущего шага в формате datetime
        utc_time = datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds)

        # Считаем положение спутника
        lon, lat, alt = orb.get_lonlatalt(utc_time)
        if lat>=min_ugol and lat<=max_ugol:
            # Создаём в шейп-файле новый объект
            # Определеяем геометрию
            track_shape.point(lon,lat)
            # и атрибуты
            track_shape.record(i, utc_string, lat, lon)

        # Не забываем про счётчики
        i += 1

    # Вне цикла нам осталось записать созданный шейп-файл на диск.
    # Т.к. мы знаем, что координаты положений ИСЗ были получены в WGS84
    # можно заодно создать файл .prj с нужным описанием

    try:
        prj = open("%s.prj" % output_shapefile.replace('.shp',''), "w")
        # Создаем переменную с описанием EPSG:4326 (WGS84)
        wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]'
        # Записываем её в файл .prj
        prj.write(wgs84_wkt)
        # И закрываем его
        prj.close()
        # Функцией save также сохраняем и сам шейп.
        track_shape.close()
        print('End of code!')
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

**Функции считывания координат из геодатасета GeoData:**

Функция определения координат группы полигонов

In [8]:
def polygon_to_coords(buff):
  buff_coords = []
  pol = []
  for i in range(len(buff)):
    poly_mapped = mapping(buff['geometry'][i])
    poly_coordinates = poly_mapped['coordinates'][0]
    pol.append([])
    for j in range(len(poly_coordinates)):
      pol[i].append([list(poly_coordinates[j])[0], list(poly_coordinates[j])[1]]) #Возможно нужно поменять 0 и 1 местами!!!

    buff_coords.append(pol[i])

  return buff_coords

Функция определения координат группы точек

In [9]:
def point_to_coords(vitok_1):
  point_coords = []
  for i in range(len(list(vitok_1['geometry']))):
    point_coords.append([mapping(list(vitok_1['geometry'])[i])['coordinates'][1], mapping(list(vitok_1['geometry'])[i])['coordinates'][0]])
  return point_coords

Функция определения координат отдельного одного полигона

In [10]:
def poly_to_coords(left_bord_2g):
  coords = []
  for i in range(len(list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0]))):
    coords.append([list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0][i])[1], list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0][i])[0]])
  return coords

**Функция получения данных о проекции Земли по EPSG**

In [11]:
def getWKT_PRJ(epsg_code):

    ## import required
    import urllib.request

    ## access projection information
    response = urllib.request.urlopen("http://spatialreference.org/ref/epsg/{0}/prettywkt/".format(epsg_code))

    ## decode the response from bytes to string
    wkt = response.read().decode('utf-8')

    ## return with no linebreaks or spaces
    return wkt.replace("\n", "").replace(" ", "")

**Функция получения точек съёмки по заданному углу визирования**

In [12]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Библиотека pyshp понадобится для создания шейп-файла


#Просчет трассы с учётом углов визирования
def ugol_visir(input_shapefile, output_shapefile_left, output_shapefile_right, ugol):
    #Вычисление расстояния от надира в км
    import math
    import numpy as np

    R = 6371302 #м
    h = 518833 #м

    gamma = math.degrees(math.asin((h+R)*math.sin(math.radians(ugol))/R)) - ugol
    L = 2*math.pi*R*gamma/360 #Перпендикуляное расстояние от точки надира в м

    lat = []
    long = []
    #Из шейп файла считываем координаты точек в надире
    nadir = gpd.read_file(input_shapefile)
    nadir = nadir.to_crs(epsg=SK) #перевести подспутниковые точки в проекцию EPSG3043
    coords_nadir = point_to_coords(nadir)
    for i in range(len(coords_nadir)):
      if coords_nadir[i][1]!= math.inf:
        lat.append(coords_nadir[i][1])
        long.append(coords_nadir[i][0])

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape1 = shapefile.Writer(output_shapefile_left)
    track_shape2 = shapefile.Writer(output_shapefile_right)

    # Добавляем поля - широту и долготу
    track_shape1.field('LAT','F',40)
    track_shape1.field('LON','F',40)
    track_shape2.field('LAT','F',40)
    track_shape2.field('LON','F',40)

    lat_new_r = []
    lat_new_l = []
    long_new_r = []
    long_new_l = []

    for k in range(len(lat)-1):
      if lat[k+1]-lat[k] ==0:
          #Восхождение
          if (long[k]<long[k+2]):
              #Правый борт
              lat_new_r.append(lat[k]+L)
              long_new_r.append(long[k])
              #Левый борт
              lat_new_l.append(lat[k]-L)
              long_new_l.append(long[k])

          #Нисхождение
          if (long[k]>long[k+2]):
              #Правый борт
              lat_new_r.append(lat[k]+L)
              long_new_r.append(long[k])
              #Левый борт
              lat_new_l.append(lat[k]-L)
              long_new_l.append(long[k])
      elif long[k+1]-long[k] ==0:
        #Восхождение
          if (lat[k]<lat[k+2]):
              #Правый борт
              lat_new_r.append(lat[k])
              long_new_r.append(long[k]+L)
              #Левый борт
              lat_new_l.append(lat[k])
              long_new_l.append(long[k]-L)

          #Нисхождение
          if (lat[k]>lat[k+2]):
              #Правый борт
              lat_new_r.append(lat[k])
              long_new_r.append(long[k]-L)
              #Левый борт
              lat_new_l.append(lat[k])
              long_new_l.append(long[k]+L)
      else:
          delt_s = abs(lat[k+1]-lat[k])
          delt_d = abs(long[k+1]-long[k])
          #Угол отклонения скорости
          beta = math.atan(delt_s/delt_d) #!

          #Восхождение
          if (lat[k]<lat[k+1]):
            #Правый борт
            lat_new_r.append(lat[k]+L*math.cos(beta))
            long_new_r.append(long[k]+L*math.sin(beta))
            #Левый борт
            lat_new_l.append(lat[k]-L*math.cos(beta))
            long_new_l.append(long[k]-L*math.sin(beta))

          #Нисхождение
          if (lat[k]>lat[k+1]):
            #Правый борт
            lat_new_r.append(lat[k]+L*math.cos(beta))
            long_new_r.append(long[k]-L*math.sin(beta))
            #Левый борт
            lat_new_l.append(lat[k]-L*math.cos(beta))
            long_new_l.append(long[k]+L*math.sin(beta))


    for i in range(len(long_new_r)):

        # Создаём в шейп-файле новый объект
        # Определеяем геометрию
        track_shape1.point(lat_new_r[i],long_new_r[i])
        # и атрибуты
        track_shape1.record(lat_new_r[i],long_new_r[i])

        track_shape2.point(lat_new_l[i],long_new_l[i])
        track_shape2.record(lat_new_l[i],long_new_l[i])

    try:
        # Создаем файл .prj с тем же именем, что и выходной .shp
        prj = open("%s.prj" % output_shapefile_left.replace('.shp',''), "w")
        ## call the function and supply the epsg code
        epsg_info = getWKT_PRJ(SK)
        prj.write(epsg_info)
        # И закрываем его
        prj.close()
        # Создаем файл .prj с тем же именем, что и выходной .shp
        prj = open("%s.prj" % output_shapefile_right.replace('.shp',''), "w")
         ## call the function and supply the epsg code
        epsg_info = getWKT_PRJ(SK)
        prj.write(epsg_info)
        # И закрываем его
        prj.close()


        # Функцией save также сохраняем и сам шейп.
        #track_shape.save(output_shapefile)
        track_shape1.close()
        track_shape2.close()
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

    print('Ready')


**Функция получения полигонов полос визирования для отдельного витка**

In [13]:
def add_vitok(input_shapefile, min_ugol, max_ugol, output_l1, output_l2, output_r1, output_r2):
  ugol_visir(input_shapefile,'min_left.shp','min_right.shp', min_ugol)
  ugol_visir(input_shapefile,'max_left.shp','max_right.shp', max_ugol)
  import warnings
  warnings.filterwarnings('ignore')

  u20_r = gpd.read_file('min_right.shp')
  u55_r = gpd.read_file('max_right.shp')
  u20_l = gpd.read_file('min_left.shp')
  u55_l = gpd.read_file('max_left.shp')

  u20_r = u20_r.to_crs(epsg=4326)
  u55_r = u55_r.to_crs(epsg=4326)
  u20_l = u20_l.to_crs(epsg=4326)
  u55_l = u55_l.to_crs(epsg=4326)

  print('Try to do polygons')
  min_l_lat = []
  min_l_lon = []
  max_l_lat = []
  max_l_lon = []

  min_r_lat = []
  min_r_lon = []
  max_r_lat = []
  max_r_lon = []
  for i in range(len(u20_r)):
    min_r_lat.append(point_to_coords(u20_r)[i][0])
    min_r_lon.append(point_to_coords(u20_r)[i][1])

  for i in range(len(u20_l)):
    min_l_lat.append(point_to_coords(u20_l)[i][0])
    min_l_lon.append(point_to_coords(u20_l)[i][1])

  for i in range(len(u55_r)):
    max_r_lat.append(point_to_coords(u55_r)[i][0])
    max_r_lon.append(point_to_coords(u55_r)[i][1])

  for i in range(len(u55_l)):
    max_l_lat.append(point_to_coords(u55_l)[i][0])
    max_l_lon.append(point_to_coords(u55_l)[i][1])

  print('Modeling')

  #Учет смены полушарий - переход через 180 меридиан
  global ind20
  ind20 = 0


  for i in range(len(u20_l)-1):
    if min_l_lon[i]>0 and min_l_lon[i-1]<0:
      ind20 = i
      break

  if (ind20==0):
    schema = {
        'geometry':'Polygon'
    }

    left_bord = fiona.open(output_l1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    for k in range(len(min_l_lon)):
      if min_l_lon[k]>=-150:
        xyList.append((min_l_lon[k], min_l_lat[k]))

    for k in range(len(max_l_lon)-1, -1, -1):
      if min_l_lon[k]>=-150:
        xyList.append((max_l_lon[k], max_l_lat[k]))

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord.write(rowDict)
    left_bord.close()
    print('Left has done')

    right_bord = fiona.open(output_r1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    for k in range(len(min_r_lon)):
      if min_r_lon[k]>=-150:
        xyList.append((min_r_lon[k], min_r_lat[k]))

    for k in range(len(max_r_lon)-1, -1, -1):
      if max_r_lon[k]>=-150:
        xyList.append((max_r_lon[k], max_r_lat[k]))
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord.write(rowDict)
    right_bord.close()
    print('Right has done')

  else:
    schema = {
        'geometry':'Polygon'
    }

    left_bord_1 = fiona.open(output_l1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=0
    while  k!=ind20-1:#!-1 изначально
      if min_l_lon[k]<=160:
        xyList.append((min_l_lon[k], min_l_lat[k]))
      k+=1

    k=ind20-2 #!
    while  k!=0-1:
      if max_l_lon[k]<=160:
        xyList.append((max_l_lon[k], max_l_lat[k]))
      k-=1

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord_1.write(rowDict)
    left_bord_1.close()
    print('Left 1 has done')

    right_bord_1 = fiona.open(output_r1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=0
    while  k!=ind20: #!!!
      if min_r_lon[k]<=160:
        xyList.append((min_r_lon[k], min_r_lat[k]))
      k+=1

    k=ind20 #!
    while  k!=0: #!
      if max_r_lon[k]<=160:
        xyList.append((max_r_lon[k], max_r_lat[k]))
      k-=1
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord_1.write(rowDict)
    right_bord_1.close()
    print('Right 1 has done')

    left_bord_2 = fiona.open(output_l2, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=ind20
    while  k!=len(u20_l)-1: #-1 изначально
      if min_l_lon[k]>=-160:
        xyList.append((min_l_lon[k], min_l_lat[k]))
      k+=1

    k=len(u20_l)-1
    while  k!=ind20: #!
      if max_l_lon[k]>=-160:
        xyList.append((max_l_lon[k], max_l_lat[k]))
      k-=1

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord_2.write(rowDict)
    left_bord_2.close()
    print('Left 2 has done')

    right_bord_2 = fiona.open(output_r2, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=ind20+1 #!!
    while  k!=len(min_r_lon)-1:
      if min_r_lon[k]>=-160:
        xyList.append((min_r_lon[k], min_r_lat[k]))
      k+=1

    k=len(min_r_lon)-1 #-1 изначально
    while  k!=ind20+1: #!+1 изначально
      if max_r_lon[k]>=-160:
        xyList.append((max_r_lon[k], max_r_lat[k]))
      k-=1
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord_2.write(rowDict)
    right_bord_2.close()
    print('Right 2 has done')

**Блок расчёта и создания полигонов полос обзора.**

In [14]:
get_spacetrack_tle (nomer, datetime(year, mounth, day), datetime(year, mounth, day+1), USERNAME, PASSWORD, latest=True)

2025-07-21


'Txt create'

In [15]:
b_d = datetime(year, mounth, day, 0, 0, 0)
b= b_d.hour+b_d.minute+b_d.second
i=vitkov
while i!=0:
    c= b+shag
    print(datetime(year, mounth, day, b//3600, (b%3600)//60, b%60))
    if c//3600 ==24:
        c=0
        #day+=1
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_north(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, 23, 59, 59), interval, papka+'/{}/vitok_{}_cut_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), min_ugol_obz, max_ugol_obz, papka + '/tle.txt')
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, 23, 59, 59), interval, papka+'/{}/vitok_{}_pol_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    else:
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_north(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, c//3600, (c%3600)//60, c%60), interval, papka+'/{}/vitok_{}_cut_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), min_ugol_obz, max_ugol_obz, papka + '/tle.txt')
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, c//3600, (c%3600)//60, c%60), interval, papka+'/{}/vitok_{}_pol_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    b=c
    i-=1
for i in range(1, vitkov+1, 1):
    add_vitok(papka+'/{}/vitok_{}_cut_{}.shp'.format(i, i, nomer), min_ugol_viz, max_ugol_viz, papka+"/{}/vitok_{}_cut_{}_l1_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_l2_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_r1_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_r2_test.shp".format(i, i, nomer))
print('Подспутниковые точки и зоны визирования сохранены в указанную папку: ' + papka)

2025-07-15 00:00:00
2025-07-15 01:00:00
End of code!
End of code!
2025-07-15 01:00:00
2025-07-15 02:00:00
End of code!
End of code!
2025-07-15 02:00:00
2025-07-15 03:00:00
End of code!
End of code!
2025-07-15 03:00:00
2025-07-15 04:00:00
End of code!
End of code!
2025-07-15 04:00:00
2025-07-15 05:00:00
End of code!
End of code!
2025-07-15 05:00:00
2025-07-15 06:00:00
End of code!
End of code!
2025-07-15 06:00:00
2025-07-15 07:00:00
End of code!
End of code!
2025-07-15 07:00:00
2025-07-15 08:00:00
End of code!
End of code!
2025-07-15 08:00:00
2025-07-15 09:00:00
End of code!
End of code!
2025-07-15 09:00:00
2025-07-15 10:00:00
End of code!
End of code!
2025-07-15 10:00:00
2025-07-15 11:00:00
End of code!
End of code!
2025-07-15 11:00:00
2025-07-15 12:00:00
End of code!
End of code!
2025-07-15 12:00:00
2025-07-15 13:00:00
End of code!
End of code!
2025-07-15 13:00:00
2025-07-15 14:00:00
End of code!
End of code!
2025-07-15 14:00:00
2025-07-15 15:00:00
End of code!
End of code!
2025-07-15

**Создание таблицы рабочей программы (время попадания точки прицеливания в полосы обзора КА) с учетом только баллистических ограничений**

In [16]:
def get_second_working_directory(a1,a2):
    root = tk.Tk()
    root.title("Условие пользователю")
    root.geometry("1000x200")
    root.resizable(False, False)

    def submit():
        root.destroy()

    folder_path = tk.StringVar()

    tk.Label(root, text = a1 , font=("Arial", 15)).pack(pady=(10, 5))

    tk.Label(root, text = a2 , font=("Arial", 15)).pack(pady=(11, 5))
    
    tk.Label(root, text = papka , font=("Arial", 14)).pack(pady=(15, 5))
    
    submit_btn = tk.Button(root, text="OK", command=submit, width=20, font=("Arial", 10))
    submit_btn.pack(pady=10)
    
    root.mainloop()
    return

In [17]:
if __name__ == "__main__":
    get_second_working_directory("Пожалуйста загрузите Shapefile точек прицеливания","с именем shooting_points.shp в папку:")

In [18]:
interval=1

b_d = datetime(year, mounth, day, 0, 0, 0)
b= b_d.hour+b_d.minute+b_d.second
i=vitkov
while i!=0:
    c= b+shag
    print(datetime(year, mounth, day, b//3600, (b%3600)//60, b%60))
    if c//3600 ==24:
        c=0
        #day+=1
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, 23, 59, 59), interval, papka + '/{}/vitok_{}_step1_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    else:
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, c//3600, (c%3600)//60, c%60), interval, papka + '/{}/vitok_{}_step1_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    b=c
    i-=1

2025-07-15 00:00:00
2025-07-15 01:00:00
End of code!
2025-07-15 01:00:00
2025-07-15 02:00:00
End of code!
2025-07-15 02:00:00
2025-07-15 03:00:00
End of code!
2025-07-15 03:00:00
2025-07-15 04:00:00
End of code!
2025-07-15 04:00:00
2025-07-15 05:00:00
End of code!
2025-07-15 05:00:00
2025-07-15 06:00:00
End of code!
2025-07-15 06:00:00
2025-07-15 07:00:00
End of code!
2025-07-15 07:00:00
2025-07-15 08:00:00
End of code!
2025-07-15 08:00:00
2025-07-15 09:00:00
End of code!
2025-07-15 09:00:00
2025-07-15 10:00:00
End of code!
2025-07-15 10:00:00
2025-07-15 11:00:00
End of code!
2025-07-15 11:00:00
2025-07-15 12:00:00
End of code!
2025-07-15 12:00:00
2025-07-15 13:00:00
End of code!
2025-07-15 13:00:00
2025-07-15 14:00:00
End of code!
2025-07-15 14:00:00
2025-07-15 15:00:00
End of code!
2025-07-15 15:00:00
2025-07-15 16:00:00
End of code!
2025-07-15 16:00:00
2025-07-15 17:00:00
End of code!
2025-07-15 17:00:00
2025-07-15 18:00:00
End of code!
2025-07-15 18:00:00
2025-07-15 19:00:00
End of

In [19]:
def nearest_subsatellite_point(input_shapefile_tpv, input_papka_pt): #входной файл с точками
    min_s=99999999999999999
    time=[]
    a=-1
    #Обработка данных о подспутниковых точках
    file_tpv = gpd.read_file(input_shapefile_tpv)
    df_tpv = pd.DataFrame(file_tpv)
    gdf_tpv = gpd.GeoDataFrame(df_tpv, geometry = gpd.points_from_xy(df_tpv.lat, df_tpv.lon))
    for l in range(1, df_tpv['vitok'].max()+1):
        file_pt = gpd.read_file(input_papka_pt+'/{}/vitok_{}_step1_{}.shp'.format(l, l, nomer))
        df_pt = pd.DataFrame(file_pt)
        gdf_pt = gpd.GeoDataFrame(df_pt, geometry = gpd.points_from_xy(df_pt.LAT, df_pt.LON))
        for j in range(len(file_tpv)): #перебор точек в полигонах
            if l==file_tpv['vitok'][j]:
                for i in range(len(file_pt)): #перебор точек в трасс
                    if min_s>((file_tpv["lat"][j]-file_pt["LAT"][i])**2+(file_tpv["lon"][j]-file_pt["LON"][i])**2):
                        min_s=(file_tpv["lat"][j]-file_pt["LAT"][i])**2+(file_tpv["lon"][j]-file_pt["LON"][i])**2
                        a=i
                time.append(file_pt["TIME"][a])
                min_s=99999999999999999
                a=-1
    print(time)
    print(len(time))
    with open(papka + "/shooting_time_all.csv", "w", newline="") as file:  
        writer = csv.writer(file)  
        writer.writerow(['TIME'])  
        for item in time:  
            writer.writerow([item])  
    print("CSV file created: shooting_time_all.csv")

In [20]:
nearest_subsatellite_point(papka + '/shooting_points.shp', papka)

['2025-07-15 00:11:34', '2025-07-15 00:09:03', '2025-07-15 01:43:18', '2025-07-15 06:41:09', '2025-07-15 08:13:05', '2025-07-15 08:13:14', '2025-07-15 08:13:31', '2025-07-15 09:45:55', '2025-07-15 11:19:44', '2025-07-15 11:22:49', '2025-07-15 11:21:39', '2025-07-15 11:20:20', '2025-07-15 12:53:31', '2025-07-15 12:57:51', '2025-07-15 12:55:47', '2025-07-15 12:54:06', '2025-07-15 14:30:21', '2025-07-15 14:32:04', '2025-07-15 14:28:39', '2025-07-15 14:29:16', '2025-07-15 15:00:00', '2025-07-15 16:06:24', '2025-07-15 16:05:02', '2025-07-15 16:04:03', '2025-07-15 16:03:12', '2025-07-15 16:02:38', '2025-07-15 16:01:50', '2025-07-15 17:31:27', '2025-07-15 17:39:51', '2025-07-15 17:39:07', '2025-07-15 17:38:41', '2025-07-15 17:37:59', '2025-07-15 17:37:12', '2025-07-15 17:36:16', '2025-07-15 17:34:49', '2025-07-15 17:33:35', '2025-07-15 19:12:41', '2025-07-15 19:11:56', '2025-07-15 19:10:42', '2025-07-15 19:06:33', '2025-07-15 19:05:10', '2025-07-15 20:46:36', '2025-07-15 20:45:41', '2025-07-1

**Расчет зон радиовидимости пунктов приёма информации, заданных точками ppi.csv**

In [21]:
if __name__ == "__main__":
    get_second_working_directory("Пожалуйста загрузите файл пунктов приёма информации","с именем ppi.csv в папку:")

In [22]:
if __name__ == "__main__":
    get_second_working_directory("Пожалуйста создайте папку с именем ЗРВ","в рабочей директории:")

In [23]:
def calculate_radius_of_zrv(min_delta_angle=5, alt=518833, radius_of_earth=6371032):
    angle_of_zone_of_radiovision = math.acos((math.cos(min_delta_angle * (math.pi / 180)) * radius_of_earth) / (alt + radius_of_earth)) - (min_delta_angle * (math.pi / 180))
    return angle_of_zone_of_radiovision * radius_of_earth

In [24]:
def genaral_zrv(ppi_id, file_input, file_output):
    PPI = pd.read_csv(file_input)
    ppi_gdf = gpd.GeoDataFrame(PPI, geometry=gpd.points_from_xy(PPI.Longitude, PPI.Latitude))
    for id, row in ppi_gdf.iterrows():
        if id not in ppi_id:
            ppi_gdf.drop(axis=0, index=id, inplace=True)
    ppi_gdf.crs="EPSG:4326"
    ppi_converted = ppi_gdf.to_crs(epsg=3043)
    ppi_converted["geometry"] = ppi_converted.buffer(calculate_radius_of_zrv())
    ppi_result = ppi_converted.to_crs(epsg=4326)
    ppi_result.to_file(file_output + '/{}/GENERAL_PPI.shp'.format('ЗРВ'))

In [25]:
def make_zrv(ppi_id=[], file_input=papka + '/ppi.csv', file_output=papka):
    PPI = pd.read_csv(file_input)
    ppi_gdf = gpd.GeoDataFrame(PPI, geometry=gpd.points_from_xy(PPI.Longitude, PPI.Latitude))
    ppi_gdf.crs = "EPSG:4326"
    ppi_gdf.to_file(file_output + '/{}/PPI_POINTS.shp'.format('ЗРВ'))
    ppi_converted = ppi_gdf.to_crs(epsg=3043)
    ppi_converted["geometry"] = ppi_converted.buffer(calculate_radius_of_zrv())
    ppi_result = ppi_converted.to_crs(epsg=4326)
    genaral_zrv(ppi_id=ppi_id, file_input=file_input, file_output=file_output)
    step = 0
    for i in range(len(ppi_result)):
        ppi_table = ppi_result
        for id, row in ppi_table.iterrows():
            if id != i:
                ppi_table = ppi_table.drop(axis=0, index=id, inplace=False)
        if i in ppi_id:
            ppi_table.to_file(file_output + '/{}/test_ppi'.format('ЗРВ') + str(step) + '.shp')
        step += 1

In [26]:
def piar(a):
    root = tk.Tk()
    root.title("Уведомление пользователю")
    root.geometry("1000x100")
    root.resizable(False, False)

    def submit():
        root.destroy()

    folder_path = tk.StringVar()

    tk.Label(root, text = a , font=("Arial", 15)).pack(pady=(10, 5))
    
    submit_btn = tk.Button(root, text="OK", command=submit, width=20, font=("Arial", 10))
    submit_btn.pack(pady=10)
    
    root.mainloop()
    return

In [27]:
make_zrv(ppi_id=list(range(12)))
# make_zrv([0, 1, 2, 3, 10, 11])
piar('Полигоны ЗРВ сохранены в папку: ' + papka+'/ЗРВ')

In [28]:
create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, 0, 0, 0), datetime(year, mounth, day, 23, 59, 59), interval, papka + '/trassirovka_{}.shp'.format(nomer), papka + '/tle.txt')

End of code!


In [29]:
if __name__ == "__main__":
    piar("Расчёт сеансов связи занимает около 10 минут, подождите!")

In [30]:
def intersect_subsatellite_points_with_zrv(input_zrv=papka + '/ЗРВ/GENERAL_PPI.shp', input_subsatellite_points=papka + '/trassirovka_{}.shp'.format(nomer)):    
    zrv_gdf = gpd.read_file(input_zrv)
    step = 0
    previous_state = 0
    ppi_session = []
    start_session = []
    end_session = []
    for id, row in zrv_gdf.iterrows():
        subsat_points_gdf = gpd.read_file(input_subsatellite_points)
        polygon_coordinates = row["geometry"]
        polygon = Polygon(polygon_coordinates)
        # start_gdf = gpd.GeoDataFrame()
        # end_gdf = gpd.GeoDataFrame()
        for id_point, subs_point in subsat_points_gdf.iterrows():
            x = float(subs_point.geometry.x)
            y = float(subs_point.geometry.y)
            point = Point(x, y)
            current_state = point.within(polygon)
            if current_state == 0 and previous_state == 0:
                subsat_points_gdf.drop(axis=0, index=id_point, inplace=True)
            elif current_state == 1 and previous_state == 0:
                start_session.append(subs_point["TIME"])
                ppi_session.append(id)
            elif current_state == 0 and previous_state == 1:
                end_session.append(subs_point["TIME"])
            previous_state = current_state
        if not subsat_points_gdf.empty:
            subsat_points_gdf.to_file(papka + '/ЗРВ/intersected' + str(step) + '.shp')
            if __name__ == "__main__":
                piar("Ещё один сеанс сохранён!")
        step += 1
    session_data = dict(PPI=ppi_session, START=start_session, END=end_session)
    session_df = pd.DataFrame(session_data)
    session_df.to_excel(papka + '/ЗРВ/SESSION_PPI.xlsx', index=False)

In [31]:
intersect_subsatellite_points_with_zrv()

**Учет временных ограничений**

In [81]:
if __name__ == "__main__":
    get_second_working_directory("Пожалуйста загрузите файл с временными ограничениями","работ ЦУПа с именем warnings.txt в папку:")

In [82]:
if __name__ == "__main__":
    get_second_working_directory("Пожалуйста создайте папку с именем CUP","в рабочей директории:")

In [65]:
def create_table_cup(input_file= papka + '/warnings.txt'):
    file = open(input_file)
    tcu = []
    opd = []
    no_info_strings = 0
    step = 0
    for i in file:
        if i[0] in '0123456789':
            parts = i.split()
            date_start_list = parts[0].split("T")
            date_end_list = parts[1].split("T")
            date_start = datetime(int(date_start_list[0][:4]), int(date_start_list[0][5:7]), int(date_start_list[0][8:10]), int(date_start_list[1][:2]), int(date_start_list[1][3:5]), int(date_start_list[1][6:8]))
            date_end = datetime(int(date_end_list[0][:4]), int(date_end_list[0][5:7]), int(date_end_list[0][8:10]), int(date_end_list[1][:2]), int(date_end_list[1][3:5]), int(date_end_list[1][6:8]))
            if no_info_strings < 5:
                tcu.append([date_start, date_end])
            else:
                opd.append([date_start, date_end])
        else:
            no_info_strings += 1
    return tcu, opd

In [66]:
def is_time_between(begin_time, end_time, check_time):
    if begin_time < end_time:
        return check_time >= begin_time and check_time <= end_time
    else:
        return check_time >= begin_time or check_time <= end_time

In [67]:
def cup_intersections(input_subsatellite_points= papka + '/trassirovka_{}.shp'.format(nomer), output_file=papka + '/CUP/'):
    subs_points = gpd.read_file(input_subsatellite_points)
    cup_tcu, cup_opd = create_table_cup()
    step = 0
    for time_interval in cup_tcu:
        cup_points = subs_points
        for id, point in subs_points.iterrows():
            point_date_time = datetime.strptime(point["TIME"], '%Y-%m-%d %H:%M:%S')
            if not is_time_between(time_interval[0], time_interval[1], point_date_time):
                cup_points = cup_points.drop(axis=0, index=id, inplace=False)
        cup_points.to_file(str(output_file) + 'tcu_points' + str(step) + '.shp')
        step += 1
    step = 0
    for time_interval in cup_opd:
        cup_points = subs_points
        for id, point in subs_points.iterrows():
            point_date_time = datetime.strptime(point["TIME"], '%Y-%m-%d %H:%M:%S')
            if not is_time_between(time_interval[0], time_interval[1], point_date_time):
                cup_points = cup_points.drop(axis=0, index=id, inplace=False)
        cup_points.to_file(str(output_file) + 'opd_points' + str(step) + '.shp')
        step += 1

In [83]:
if __name__ == "__main__":
    piar("Расчёт участков трассировки с ограничениями занимает около 10 минут, подождите!")

In [70]:
cup_intersections()

In [71]:
def success_shooting(time_delta=81, shooting_programm=papka + '/shooting_time_all.csv', correct_shooting_program=papka + '/corrected_shooting_points_1.xlsx'):
    tcu, opd = create_table_cup()
    restricts = tcu + opd
    shooting_input = pd.read_csv(shooting_programm)
    shooting_table = pd.DataFrame(shooting_input)
    dates_of_shooting = []
    for id, row in shooting_table.iterrows():
        dates_of_shooting.append(datetime.strptime(row["TIME"], '%Y-%m-%d %H:%M:%S'))
    dates_of_shooting.sort()
    is_valid_coloumn = []
    time_delta_valid = []
    reason_of_false = []
    last_date_of_shooting = None
    # print('START\t\t\tEND\t\t\tCHECK\t\t\tSTATE')
    for row in dates_of_shooting:
        all = 0
        reason = None
        for i in restricts:
            if not is_time_between(i[0], i[1], row):
                all += 1
            else:
                reason = "TCU" if i in tcu else "OPD"
            # print(str(i[0]) + '\t' + str(i[1]) + '\t' + str(row) + '\t' + str(True if not is_time_between(i[0], i[1], row) else False))
        
        if all == len(restricts):
            is_valid_coloumn.append(True)
            reason_of_false.append(None)
        else:
            is_valid_coloumn.append(False)
            reason_of_false.append(reason)
        try:
            if int((row - last_date_of_shooting).total_seconds()) <= time_delta:
                # time_delta_valid[-1] = False
                time_delta_valid.append(False)
            else:
                time_delta_valid.append(True)
                last_date_of_shooting = row
        except:
            time_delta_valid.append(True)
            last_date_of_shooting = row
    is_valid_series = pd.Series(is_valid_coloumn, name="IS_VALID")
    time_delta_valid_series = pd.Series(time_delta_valid, name='TM_DLT_VLD')
    shooting_table["TIME"] = dates_of_shooting
    shooting_table["IS_VALID"] = is_valid_series
    shooting_table["TM_DLT_VLD"] = time_delta_valid_series
    shooting_table["REASON"] = reason_of_false
    shooting_table.to_excel(correct_shooting_program, index=False)

In [73]:
success_shooting()

**Блок ограничения памяти. Составление таблицы целевой работы КА: съёмка, запись в ЗУ, сброс на ППИ, удаление кадра из ЗУ.**

In [85]:
if __name__ == "__main__":
    piar("Началось составление итоговой таблицы целевой работы КА")

In [74]:
def is_time_between(begin_time, end_time, check_time):
    if begin_time < end_time:
        return check_time >= begin_time and check_time <= end_time
    else:
        return check_time >= begin_time or check_time <= end_time

In [75]:
def intersect_sessions(first, second, switch_mode_time=60, drop_speed=350, one_image_volume=1000):
    start1 = first[0]
    end1 = first[1]
    start2 = second[0]
    end2 = second[1]
    if int((start2 - end1).total_seconds()) < switch_mode_time:
        if (start1 == start2) and (end1 == end2):
            return 0
        elif (start1 < start2) and (end2 < end1):
            return 0
        else:
            start2 = end1 + timedelta(seconds=switch_mode_time)
            if int((end2 - start2).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                return [start2, end2]
            else:
                return 0
    else:
        return [start2, end2]

In [76]:
def intersect_capture_with_sessions(capture, session, future_session=True, capture_time=15, switch_mode_time=60, drop_speed=350, one_image_volume=1000):
    start_cap = capture[0]
    end_cap = capture[1]
    start_session = session[0]
    end_session = session[1]
    if future_session:
        if int((start_session - end_cap).total_seconds()) < switch_mode_time:
            if (start_cap == start_session) and (end_cap == end_session):
                return 0
            elif (start_cap < start_session) and (end_session < end_cap):
                return 0
            else:
                start_session = end_cap + timedelta(seconds=switch_mode_time)
                if int((end_session - start_session).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                    return [start_session, end_session]
                else:
                    return 0
        else:
            return [start_session, end_session]
    else:
        if int((start_cap - end_session).total_seconds()) < switch_mode_time:
            if (start_cap == start_session) and (end_cap == end_session):
                return 0
            elif (start_session < start_cap) and (end_cap < end_session):
                after = int((end_session - end_cap).total_seconds()) - switch_mode_time
                before = int((start_cap - start_session).total_seconds()) - switch_mode_time
                if after >= before:
                    if after >= int((one_image_volume * 8) / (drop_speed)):
                        return [end_cap + timedelta(seconds=switch_mode_time), end_session]
                    else:
                        return 0
                else:
                    if before >= int((one_image_volume * 8) / (drop_speed)):
                        return [start_session, start_cap - timedelta(seconds=switch_mode_time)]
                    else:
                        return 0                
            else:
                end_session = start_cap - timedelta(seconds=switch_mode_time)
                if int((end_session - start_session).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                    return [start_session, end_session]
                else:
                    return 0
        else:
            return [start_session, end_session]

In [77]:
def calculate_memory_restricts(memory_store, cell, drop_speed=350, capture_time=15, switch_mode_time=60, one_image_volume=1000, shooting_program_file=papka + '/corrected_shooting_points_1.xlsx', ppi_sessions_file=papka +'/ЗРВ/SESSION_PPI.xlsx', output_file=papka + '/result_data.xlsx'):
    connection_sessions = pd.read_excel(ppi_sessions_file)
    shooting_sessions = pd.read_excel(shooting_program_file)
    
    # Удаляем из DataFrame те съемки, которые пересекаются с огр. ЦУПа или находятся в интервале 66 секунд после предыдущей съемки
    for id, row in shooting_sessions.iterrows():
        if (not bool(row["IS_VALID"])) or (not bool(row["TM_DLT_VLD"])):
            shooting_sessions = shooting_sessions.drop(axis=0, index=id, inplace=False)
    
    drop_plan = []
    shooting_plan = []
    total_plan = []   
    for id, row in shooting_sessions.iterrows():
        capture_point_time = datetime.strptime(str(row["TIME"]), '%Y-%m-%d %H:%M:%S')
        shooting_plan.append([capture_point_time, capture_point_time + timedelta(seconds=capture_time)])
    
    # Составляем план сброса информации в порядке возрастания
    for id_ses, session in connection_sessions.iterrows():
        start_session = datetime.strptime(session["START"], '%Y-%m-%d %H:%M:%S')
        end_session = datetime.strptime(session["END"], '%Y-%m-%d %H:%M:%S')
        drop_plan.append([start_session, end_session])
    drop_plan.sort(key=lambda x: x[0])

    # План сессий без пересечений
    correct_drop_plan = [drop_plan[0]]
    for i in range(len(drop_plan)):  
        try:
            correct_drop_plan.append(intersect_sessions(drop_plan[i], drop_plan[i + 1], switch_mode_time=switch_mode_time, drop_speed=drop_speed, one_image_volume=one_image_volume))
        except:
            pass
    
    correct_drop_plan2 = [drop_plan[0]]
    for i in range(len(correct_drop_plan)):
        try:
            correct_drop_plan2.append(intersect_sessions(correct_drop_plan[i], correct_drop_plan[i + 1], switch_mode_time=switch_mode_time, drop_speed=drop_speed, one_image_volume=one_image_volume))
        except:
            pass
    
    # Общий план: съемка + сессии связи
    total_plan = correct_drop_plan2 + shooting_plan
    total_plan.sort(key=lambda x: x[0])
    
    # Проверка на пересечения съемок и сессий связи
    for _ in range(3):
        for i in range(len(total_plan)):
            if total_plan[i] != 0 and total_plan[i] in shooting_plan:
                try:
                    if total_plan[i - 1] not in shooting_plan:
                        total_plan[i - 1] = intersect_capture_with_sessions(total_plan[i], total_plan[i - 1], False)
                except:
                    pass
                try:
                    if total_plan[i + 1] not in shooting_plan:
                        total_plan[i + 1] = intersect_capture_with_sessions(total_plan[i], total_plan[i + 1], True)
                except:
                    pass
        for i in total_plan:
            if i == 0:
                total_plan.remove(0)
        total_plan.sort(key=lambda x: x[0])
    
    # print('CAPTURE: ' + str(len(shooting_plan) * 15))
    # res = 0
    # for i in range(len(total_plan)):
    #     if total_plan[i] not in shooting_plan:
    #         res += int((total_plan[i][1] - total_plan[i][0]).total_seconds())
    # print('DROP: ' + str(res))

    # for i in range(len(total_plan)):
        # print(str(total_plan[i]) + '\t --- ' + str('CAPTURE' if int((total_plan[i][1] - total_plan[i][0]).total_seconds()) == 15 else 'DROP'))
    
    time_start_process_column = []
    time_end_process_column = []
    process_column = []
    # Симуляция записи и сброса информации в ЗУ КА
    for i in range(len(total_plan)):
        # print(str(total_plan[i]) + '\t --- ' + str('CAPTURE' if total_plan[i] in shooting_plan else 'DROP'))
        if total_plan[i] in shooting_plan:
            if cell == (len(memory_store) - 16):
                # print('write cell 240')
                for o in range(15):
                    if memory_store[o] == 1:
                        # print(f'WARNING: IMAGE IN {o} CELL WAS DELETED, BUT NOT DROPPED')
                        process_column.append(f'INVALID DELETE CELL {o}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    elif memory_store[o] == 2:
                        # print(f'deleted cell {o}')
                        process_column.append(f'VALID DELETE CELL {o}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)                        
                    memory_store[o] = 0
                if memory_store[cell + 14] == 1:
                    # print(f'WARNING: IMAGE IN {i + 14} CELL WAS DELETED, BUT NOT DROPPED')
                    process_column.append(f'INVALID DELETE CELL {cell + 14}')
                    time_start_process_column.append(None)
                    time_end_process_column.append(None)
                elif memory_store[cell + 14] == 2:
                    # print(f'deleted cell {cell + 14}')
                    process_column.append(f'VALID DELETE CELL {cell + 14}')
                    time_start_process_column.append(None)
                    time_end_process_column.append(None)
                memory_store[cell + 14] = 0
                
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL {cell}')
            elif cell == len(memory_store):
                # print('write cell 0')
                cell = 0
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL 0')
            else:
                # print(f'write cell {cell}')
                try:
                    if memory_store[cell + 14] == 1:
                        # print(f'WARNING: IMAGE IN {i + 14} CELL WAS DELETED, BUT NOT DROPPED')
                        process_column.append(f'INVALID DELETE CELL {cell + 14}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    elif memory_store[cell + 14] == 2:
                        # print(f'deleted cell {cell + 14}')
                        process_column.append(f'VALID DELETE CELL {cell + 14}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    memory_store[cell + 14] = 0
                except:
                    pass
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL {cell}')
            cell += 1
        else:
            for _ in range((int((total_plan[i][1] - total_plan[i][0]).total_seconds()) * drop_speed) // (one_image_volume * 8)):
                for o in range(len(memory_store)):
                    if memory_store[o] == 1:
                        memory_store[o] = 2
                        # print(f'drop cell {o}')
                        time_start_process_column.append(str(total_plan[i][0]))
                        time_end_process_column.append(str(total_plan[i][1])) 
                        process_column.append(f'DROP CELL {o}')
                        break
    data = dict(PROCESS=process_column, START=time_start_process_column, END=time_end_process_column)
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    return [memory_store, cell]

In [78]:
# 0 - no data, 1 - written data, 2 - dropped data
store = [0] * 256
current_cell = 0

In [80]:
store, current_cell = calculate_memory_restricts(store, current_cell)
print(store)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [87]:
if __name__ == "__main__":
    get_second_working_directory("Итоговая таблица целевой работы КА result_data.xlsx","сохранена в рабочей директории:")

**Расчёт ограничений средневиткового энергопотребления**

In [47]:
if __name__ == "__main__":
    piar("Начался расчёт энергопотребления")

In [48]:
def create_second_input_dialog():
    """Создает и возвращает диалоговое окно для ввода параметров"""
    root = tk.Tk()
    root.title("Ввод параметров")
    root.resizable(False, False)
    
    # Словарь для хранения виджетов и значений
    entries = {}
    values = {
        'mosnostsred': tk.StringVar(),
        'mosnostvdez': tk.StringVar(),
        'mosnost': tk.StringVar(),
        'prodolz': tk.StringVar(),
        'period': tk.StringVar()
    }
    
    # Функция проверки и сохранения данных
    def submit():
        errors = []
        # Проверка числовых полей
        numeric_fields = [
            'prodolz','period'
        ]
        
        for field in numeric_fields:
            try:
                int(values[field].get())
            except ValueError:
                errors.append(f"Некорректное значение в поле '{field_labels[field]}'")
        
        if errors:
            messagebox.showerror("Ошибки ввода", "\n".join(errors))
            return
        
        root.destroy()
    
    # Поля и их подписи
    field_labels = {
        'mosnostsred': "Введите установленную средневитковую мощность:",
        'mosnostvdez': "Введите мощность обеспечивающих систем в дежурном режиме:",
        'mosnost': "Введите мощность радиолокатора:",
        'prodolz': "Введите время продолжительности съёмки (в сек):",
        'period': "Период обращения КА (в сек):"
    }
    
    # Создание элементов формы
    for i, field in enumerate(field_labels):
        # Подпись
        lbl = tk.Label(root, text=field_labels[field], anchor='e')
        lbl.grid(row=i, column=0, padx=5, pady=5, sticky='e')
        
        # Поле ввода
        entry = tk.Entry(root, textvariable=values[field])
        
        entry.grid(row=i, column=1, padx=5, pady=5, sticky='we')
        entries[field] = entry
    
    # Кнопки управления
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=len(field_labels), column=0, columnspan=3, pady=10)
    
    btn_submit = tk.Button(btn_frame, text="OK", command=submit)
    btn_submit.pack(side=tk.LEFT, padx=10)
    
    btn_cancel = tk.Button(btn_frame, text="Отмена", command=root.destroy)
    btn_cancel.pack(side=tk.LEFT, padx=10)
    
    # Настройка размеров колонок
    root.columnconfigure(0, weight=0)
    root.columnconfigure(1, weight=1)
    root.columnconfigure(2, weight=0)
    
    root.mainloop()
    
    # Возвращаем результаты после закрытия окна
    if all(values[field].get() for field in values):
        return {k: v.get() for k, v in values.items()}
    return None

In [49]:
if __name__ == "__main__":
    # Вызываем диалоговое окно
    input_data = create_second_input_dialog()
    P_ust = float(input_data['mosnostsred'])
    P_ob = float(input_data['mosnostvdez'])
    N = float(input_data['mosnost'])
    t = int(input_data['prodolz'])
    T = int(input_data['period'])

In [50]:
def sitog(a):
    """Создает и возвращает диалоговое окно для ввода параметров"""
    root = tk.Tk()
    root.title("Ввод")
    root.resizable(False, False)
    
    # Словарь для хранения виджетов и значений
    entries = {}
    values = {
        a : tk.StringVar(),
    }
    
    # Функция проверки и сохранения данных
    def submit():        
        root.destroy()
    
    # Поля и их подписи
    field_labels = {
        a : a,
    }
    
    # Создание элементов формы
    for i, field in enumerate(field_labels):
        # Подпись
        lbl = tk.Label(root, text=field_labels[field], anchor='e')
        lbl.grid(row=i, column=0, padx=5, pady=5, sticky='e')
        
        # Поле ввода
        entry = tk.Entry(root, textvariable=values[field])
        
        entry.grid(row=i, column=1, padx=5, pady=5, sticky='we')
        entries[field] = entry
    
    # Кнопки управления
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=len(field_labels), column=0, columnspan=3, pady=10)
    
    btn_submit = tk.Button(btn_frame, text="OK", command=submit)
    btn_submit.pack(side=tk.LEFT, padx=10)
    
    btn_cancel = tk.Button(btn_frame, text="Отмена", command=root.destroy)
    btn_cancel.pack(side=tk.LEFT, padx=10)
    
    # Настройка размеров колонок
    root.columnconfigure(0, weight=0)
    root.columnconfigure(1, weight=1)
    root.columnconfigure(2, weight=0)
    
    root.mainloop()
    
    # Возвращаем результаты после закрытия окна
    if all(values[field].get() for field in values):
        return {k: v.get() for k, v in values.items()}[a]
    return None

In [62]:
def energy_consumption(P_ust, P_obes, N_radio, T_KA, t_sem, input_csv):
    '''file=gpd.read_file(input_csv)
    df=pd.DataFrame(file)'''
    df = pd.read_csv(input_csv)
    nv=[]
    str=[]
    time_sec=0
    P_sem=0
    P_mv=0
    P_no=0
    P_o=0
    for i in range(len(df)):
        str=df['TIME'][i]
        time_sec=int(str.split()[1].split(sep=':')[0])*3600+int(str.split()[1].split(sep=':')[1])*60+int(str.split()[1].split(sep=':')[2])
        nv.append(1+(time_sec//T_KA))
    actuals_s = pd.Series(nv)
    df['VITOK'] = actuals_s
    j=0
    for i in range(1, df['VITOK'].max()+1):
        k=0
        P_sem=0
        while j<len(df) and i==df['VITOK'][j]:
            j+=1
            k+=1
        P_sem=k*N_radio*t_sem/T_KA
        P_sem=round(P_sem, 3)
        P_mv=0
        P_no=0
        P_o=0
        n=int(sitog('Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций'))
        if n!=0:
            for g in range(n):
                k=int(sitog('Введите количество систем участвующих в {}-й операции'.format(g+1)))
                for f in range(k):
                    N_k=float(sitog('Введите энергопотребление {}-й системы'.format(f+1)))
                    t_k=int(sitog('Введите время работы {}-й системы'.format(f+1)))
                    P_no+=N_k*t_k
        n=int(sitog('Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций'))
        if n!=0:
            for g in range(n):
                k=int(sitog('Введите количество систем участвующих в {}-й операции'.format(g+1)))
                for f in range(k):
                    N_k=float(sitog('Введите энергопотребление {}-й системы'.format(f+1)))
                    t_k=int(sitog('Введите время работы {}-й системы'.format(f+1)))
                    P_o+=N_k*t_k
        P_mv=P_obes+P_no+P_o+P_sem
        piar('Энергопотребление на {} витке: {} Вт'.format(i, P_mv))
        if P_mv<=P_ust:
            piar('{} - OK'.format(i))
        else:
            piar('{} - энергопотребление превышено'.format(i))

In [63]:
energy_consumption(P_ust, P_ob, N, T, t, papka + '/shooting_time_all.csv')